In [1]:
from models.rnn.rnn_plain import RNN_Plain
from models.rnn.rnn_attention import RNN_Attention

from lang import load_data_int_seq
from utils import accuracy_score
%load_ext autoreload
%autoreload 2

C:\Users\danie\anaconda3\envs\drlnd\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

## Plain RNN

In [3]:
algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 2, input_size=input_lang.n_words, hidden_size=256, output_size=output_lang.n_words, embedding_size = 28, calc_magnitude_on=False)

input_size
2564


In [7]:
algo.train(input_lang, output_lang, train)

torch.Size([9, 1])
9
input_tensor[ei]
tensor([5])
===> Encoder Input
input
tensor([5])
<=== Encoder Input
tensor([[[-0.0538, -0.6164,  1.0345, -0.1548,  0.9399, -0.1107,  0.4394,
          -1.4853, -0.3117, -0.6009,  0.2666, -0.6184,  0.6373,  0.7697,
          -0.0951, -0.1351,  1.7434, -0.7814, -0.8371, -0.6605, -0.5779,
          -0.4593, -1.9816,  0.5197,  0.3522,  1.2072, -0.2372, -0.2047]]],
       grad_fn=<ViewBackward>)
input_tensor[ei]
tensor([10])
===> Encoder Input
input
tensor([10])
<=== Encoder Input
tensor([[[ 0.8958,  0.2068, -1.8497,  1.0690, -0.4614, -0.4627, -1.8680,
           0.7912,  1.4112, -1.1326,  0.0558, -1.0916,  0.1323,  0.3884,
           0.5784, -0.9356, -1.6586,  0.4249, -1.0487,  0.7925,  0.9011,
           1.2623, -0.0264,  1.2575,  2.4449,  1.4748, -0.3584, -0.5302]]],
       grad_fn=<ViewBackward>)
input_tensor[ei]
tensor([11])
===> Encoder Input
input
tensor([11])
<=== Encoder Input
tensor([[[ 1.1099,  0.4308, -1.4962,  0.5297,  0.2892,  0.0924,  0.8

In [ ]:
pred = algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

In [8]:
print(',' in '-7,-6,-5,-4,-3,-2,-1,0')

True


In [9]:
print('-7,-6,-5,-4,-3,-2,-1,0' is not '')

True


### Plain RNN with Magnitude loss

In [ ]:
magn_algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
magn_algo.train(input_lang, output_lang, train)

In [ ]:
pred = magn_algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

##  Attention-based

In [ ]:
algo_attn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=False)
algo_attn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Attention with magnitude loss

In [ ]:
algo_attn_magn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
algo_attn_magn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn_magn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])